<a href="https://colab.research.google.com/github/AlexeyK12/Data_scientist-Sberuniversity/blob/main/%D0%94%D0%97_Regression%26Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя работа 2

## Регрессия

С 1ого октября отменяется НДС на бриллианты, теперь они становятся новым инвестиционным инструментом. Давайте сделаем модель ценообразования для них.

Скачайте датасет diamonds.csv

В нем представлены характеристики бриллиантов и их цены с сайта jamesallen (B2C площадка) с 2022-07-01

**Описание полей**


* fluor - флуорисценуия (свойство камня светиться в лучах ультр)
* symmetry - показатель симметричности
* platform - название платформы, где был размещен камень
* quality_group - составной показатель из cut polish symmetry
* size_group - каратно весовая группа
* big_size_group - каратно-весовая группа
* shape - форма
* color - цвет
* clarity - чистота
* cut - качество огранки (может быть только у круглых камней)
* polish - полировка
* id - номер камня
* date - дата
* price - цена
* carat - кол-во карат
* tablepercent - размер площадки по отношению ширине
* price_per_carat - цена за карат
* z - длина (diameter)
* x - ширина
* depth_perc - отношение высоты к ширине
* y - высота



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

%autosave 60
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
pd.options.display.max_columns = None

from IPython.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

Autosaving every 60 seconds


from google.colab import drive
drive.mount('/content/drive')

path = 'drive/MyDrive/'

In [ ]:
df = pd.read_csv('/content/diamonds.csv', index_col=0)

Необходимо предсказать `price_per_carat`

### Задание 1: Очистка

Не все камни успевают продаться за один месяц, поэтому в таблице есть повторы. Объедините данные по одному камню: подумайте, как лучше это сделать, какую цену брать.

**Попробуйте найти аномалии:** вдруг цена на некоторые камни сильно меняется (то есть продавец сам не знает, по какой цене их продавать). Также убедитесь, что остальные параметры камня не меняются.

In [ ]:
#id_stat = df.groupby(by='id')['carat'].count().to_frame() #Посчитайте статистику по кол-ву камней
id_stat = df.groupby(by='id', as_index=False)['carat'].count()

# создаём списки с несколько раз встречающимися камнями
list_id_stat_1 = id_stat.query('carat == 1')['id'].to_list()
list_id_stat_2_3 = id_stat.query('carat in [2,3]')['id'].to_list()

# разделите выборку на две части

df_one = df.query('id in @list_id_stat_1')  # те камни, которые встречались один раз
df_three = df.query('id in @list_id_stat_2_3') # те камни, которые встречались 2 или 3 раза


stat = df_three.groupby('id')['carat', 'price', 'x', 'y', 'z'].agg(np.std) # Посчитайте дстандартное отклонение по нескольким полям,
#  подумайте где оно должно равняться 0, а где меняться в каких-то разумных пределах

# список id с нулевой дисперсией по размеру
var_0_list = stat.query('x == 0 & y == 0 & z == 0').reset_index()['id'].to_list()

# Удалите аномальные наблюдения
df_three = df_three.query('id in @var_0_list').sort_values(by=['id','date'])

#тепреь в качестве цены возьмем последнее значение по времени
df_three_group = df_three.groupby(['id','price'], as_index=False).agg({'date':'max'})[['id', 'price']]

# сделаем срез из df с последней ценой
df_three = df_three_group.merge(df, how='left', on=['id', 'price'])

df = pd.concat([df_one, df_three]).reset_index(drop=True) # соединяем результаты

Цены на бриллианты достаточно сильно меняются, попробуйте вычислить коэффициент инфляции и привести цены к последнему месяцу. Стоит учесть, что цены на все бриллианты не изменяются синхронно, то есть изменение в определенных группах может быть разным.

*Определите эти группы и рассчитайте коэффициенты инфляции для каждой из них. Подправьте цены на эти коэффициенты и создайте новую переменную.*

In [ ]:
categories = ['shape','color','clarity'] #определите по каким группам отпределять инфляцию (к примеру можно добавить каратную группу)

df_index = df.groupby(['date'] + categories)[['price_per_carat']]\
             .mean()\
             .reset_index() # индекс цен

date_max = df_index.date.max()


df_index = df_index.merge(df_index.query('date == @date_max')[categories + ['price_per_carat']]\
                          .rename(columns={'price_per_carat': 'price_per_carat_max'}),
                          on=categories, how='outer') # сопоставляем группы с максимальной датой

df_index['inflation'] = df_index['price_per_carat_max'] / df_index.price_per_carat # вычисляем инфляцию

df_with_inf = df.merge(df_index[['date'] + categories + ['inflation']],
                        on=['date'] + categories, how='left') # соединяем все в одной таблице

### Задание 2: Модель

Определите функцию потерь (MSE или MAE) и аргументируйте выбор.
Попробуйте сделать baseline.
Используйте LableEncoder для категориальных фичей и **постройте линейную модель**.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

# выделим и закодируем категориальные признаки
data = df.copy()
categorical_features = ['fluor', 'symmetry', 'platform', 'shape', 'color', 'clarity', 'cut', 'polish']
data[categorical_features] = data[categorical_features].apply(LabelEncoder().fit_transform)

# выделяем целевую переменную и разделяем на обучающую и валидационную выборку
X = data.drop(columns=["price_per_carat"])
y = data["price_per_carat"]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

# нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_valid)

# обучаем модель и делаем прогноз
model_lr = LinearRegression()
model_lr.fit(X_train_scaled, y_train)
y_pred = model_lr.predict(X_test_scaled)

print('MAE:', mean_absolute_error(y_valid, y_pred))
print('MSE:', mean_squared_error(y_valid, y_pred))

MAE: 1309.7080044566283
MSE: 23944022.08034169


Теперь попробуйте OHE или TargetEncoder (сравните их).

Нормализуйте данные.

Поработайте с пропусками (обратите внимание на то, что у fluor возможен пропуск значения, а возможно отсутствие флуоресценции)

Покажите, насколько получилось улучшить результат.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

data = df.copy()
categorical_features = ['fluor', 'symmetry', 'platform', 'shape', 'color', 'clarity', 'cut', 'polish']

# кодируем категориальные признаки OneHotEncoder
encoder = OneHotEncoder()
encoded_columns = encoder.fit_transform(data[categorical_features]).toarray()
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(categorical_features))
data_encoded = pd.concat([data.drop(categorical_features, axis=1), encoded_df], axis=1)

# выделяем целевую переменную и разделяем на обучающую и валидационную выборку
X = data_encoded.drop(columns=["price_per_carat"])
y = data_encoded["price_per_carat"]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

# нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_valid)

# обучаем модель и делаем прогноз
model_lr = LinearRegression()
model_lr.fit(X_train_scaled, y_train)
y_pred = model_lr.predict(X_test_scaled)

print('MAE:', mean_absolute_error(y_valid, y_pred))
print('MSE:', mean_squared_error(y_valid, y_pred))

MAE: 922.2698416751429
MSE: 21629190.221553292


Поскольку целевой переменной мы выбрали цену, то нам крайне не выгодны большие ошибки, следовательно лучше использовать MSE, которая сильнее штрафует за большие ошибки и является для нашего случая более предпочтительной метрикой. Соответственно модель отработавшая по данным закодированным LabelEncoder с ошибкой - **MSE: 23944022** хуже, чем на данных с использованием OneHotEncoder и ошибкой - **MSE: 21629190**.

Сравните работу выбранной вами линейной модели с KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# выделяем целевую переменную и разделяем на обучающую и валидационную выборку
X = data_encoded.drop(columns=["price_per_carat"])
y = data_encoded["price_per_carat"]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

# нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_valid)

# Создание и обучение модели KNeighborsRegressor
knn_regressor = KNeighborsRegressor(n_neighbors=10)
knn_regressor.fit(X_train_scaled, y_train)

# Прогноз на тестовой выборке
y_pred = knn_regressor.predict(X_test_scaled)

print('MAE:', mean_absolute_error(y_valid, y_pred))
print('MSE:', mean_squared_error(y_valid, y_pred))

MAE: 872.4696932134096
MSE: 25871173.8301686


Модель на алгоритме к-ближайших соседей отработала хуже по метрике **MSE: 25871173**, но лучше предыдущих линейных моделей по метрике **MAE: 872.47**

## Классификация

Загрузите датасет bodyPerformance

Описание:

https://www.kaggle.com/datasets/kukuroo3/body-performance-data

from google.colab import drive
drive.mount('/content/drive')

path = 'drive/MyDrive/'

In [ ]:
df = pd.read_csv('/content/bodyPerformance.csv') #укажите свой путь

### Задание 1: Определение гендера

Постройте модель, которая будет определять гендер.

Для этого определите две метрики качества, на которые вы будете смотреть. Аргументируйте свой выбор.

После чего преобразуйте категориальные переменные и постройте модель бинарной классификации на основе линейного классификатора и сравните с KNN.

P.S.: не забудте про нормализацию.

In [ ]:
df['gender'].value_counts()

M    8467
F    4926
Name: gender, dtype: int64

Для задачи определения гендера на основе предоставленных признаков(у нас дисбаланс классов), выберем:
- **Accuracy** - общее количество правильных предсказаний, для оценки общей производительности модели;
- **F1_score** - подходит для сбалансированных и несбалансированных классов и является хорошей метрикой, когда важны и точность, и полнота.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

categorical_features = ['gender', 'class']
df[categorical_features] = df[categorical_features].apply(LabelEncoder().fit_transform)

X = df.drop(columns=["gender"])
y = df["gender"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

# нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_valid)

model_log_reg = LogisticRegression()
model_log_reg.fit(X_train_scaled, y_train)
y_pred = model_log_reg.predict(X_test_scaled)

# Оценка результатов
accuracy = accuracy_score(y_valid, y_pred)
f1 = f1_score(y_valid, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")

Accuracy: 0.9899
F1-Score: 0.9921


- модель правильно классифицировала 98,99% примеров в тестовой выборке - это высокий процент правильных ответов, что может свидетельствовать о хорошей производительности модели в целом;
-  F1-Score равное 0.9921 также является высоким - это означает, что модель демонстрирует хороший баланс между точностью и полнотой в классификации гендера.

### Задание 2: Определение класса

Теперь опробуем построить модель, которая будет предсказывать class физической формы.

Для этого определите метрики качества для задачи мультиклассификации (аргументируйте выбор).

Постройте модель мультиклассовой классификации на основе линейного классификатора и сравните с KNN.

In [ ]:
df = pd.read_csv('/content/bodyPerformance.csv') #укажите свой путь

In [ ]:
df

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...
13388,25.0,M,172.1,71.80,16.2,74.0,141.0,35.8,17.4,47.0,198.0,C
13389,21.0,M,179.7,63.90,12.1,74.0,128.0,33.0,1.1,48.0,167.0,D
13390,39.0,M,177.2,80.50,20.1,78.0,132.0,63.5,16.4,45.0,229.0,A
13391,64.0,F,146.1,57.70,40.4,68.0,121.0,19.3,9.2,0.0,75.0,D


In [ ]:
df['class'].value_counts()

C    3349
D    3349
A    3348
B    3347
Name: class, dtype: int64

In [ ]:
categorical_features = ['gender', 'class']
df[categorical_features] = df[categorical_features].apply(LabelEncoder().fit_transform)

X = df.drop(columns=["class"]) # укажите новый таргет
y = df["class"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

# нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Обучение модели логистической регрессии
model_log_reg = LogisticRegression(max_iter=50, random_state=13)
model_log_reg.fit(X_train_scaled, y_train)
y_pred_log_reg = model_log_reg.predict(X_valid_scaled)


# Обучение модели KNN
model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(X_train_scaled, y_train)
y_pred_knn = model_knn.predict(X_valid_scaled)


# Оценка результатов для логистической регрессии с средневзвешанным параметром усреднения
print("Логистическая регрессия:")
print("Accuracy:", round(accuracy_score(y_valid, y_pred_log_reg), 3))
print("Precision:", round(precision_score(y_valid, y_pred_log_reg, average="weighted"), 3))
print("Recall:", round(recall_score(y_valid, y_pred_log_reg, average="weighted"), 3))

# Оценка результатов для KNN
print('')
print("K-ближайших соседей:")
print("Accuracy:", round(accuracy_score(y_valid, y_pred_knn), 3))
print("Precision:", round(precision_score(y_valid, y_pred_knn, average="weighted"), 3))
print("Recall:", round(recall_score(y_valid, y_pred_knn, average="weighted"), 3))

Логистическая регрессия:
Accuracy: 0.615
Precision: 0.616
Recall: 0.615

K-ближайших соседей:
Accuracy: 0.591
Precision: 0.617
Recall: 0.591


Логистическая регрессия и K-ближайших соседи имеют схожую точность общей классификации, но различаются по точности и полноте. Логистическая регрессия имеет более высокую точность общей классификации и полноту, чем K-ближайшие соседи, что может означать, что она лучше подходит для задачи классификации класса физической формы.